In [6]:
import numpy as np
import pyaudio
import time
import threading

SAMPLE_RATE = 44100
INPUT_CHANNELS = 1

'''
This class is a template class for a thread that reads in audio from PyAudio.
'''

class AudioThread(threading.Thread):
    def __init__(self, 
                 name, 
                 starting_chunk_size, 
                 process_func, 
                 args_before = (), 
                 args_after = ()):
        """
        Initializes an AudioThread.
        Parameters:
            name: the name of the thread
            starting_chunk_size: an integer representing the chunk size in samples
            process_func: the function to be called as a callback when new audio is received from PyAudio
            args_before: a tuple of arguments for process_func to be put before the sound array
            args_after: a tuple of arguments for process_func to be put after the sound array
        Returns: nothing
        """
        super(AudioThread, self).__init__()
        self.name = name    # General imports
        self.process_func = process_func
        self.args_before = args_before
        self.args_after = args_after
        
        self.p = None    # PyAudio vals
        self.stream = None
        self.FORMAT = pyaudio.paFloat32
        self.CHANNELS = INPUT_CHANNELS
        self.RATE = SAMPLE_RATE
        self.CHUNK = starting_chunk_size * 2

        self.max_time = 0    # Data storage and analytics
        self.data = None
        
    def set_args_before(a):
        """
        Changes the arguments before the sound array when process_func is called.
        Parameters: a: the arguments
        Returns: nothing
        """
        self.args_before = a
    
    def set_args_after(a):
        """
        Changes the arguments after the sound array when process_func is called.
        Parameters: a: the arguments
        Returns: nothing
        """
        self.args_after = a
    
    def run(self):
        """
        When the thread is started, this function is called which opens the PyAudio object
        and keeps the thread alive.
        Parameters: nothing
        Returns: nothing
        """
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(format=self.FORMAT,
                                  channels=self.CHANNELS,
                                  rate=self.RATE,
                                  input=True,
                                  output=False,
                                  stream_callback=self.callback,
                                  frames_per_buffer=self.CHUNK)
        while (self.is_alive()):
                time.sleep(1.0)
            
    def stop(self):
        """
        When the thread is stopped, this function is called which closes the PyAudio object
        Parameters: nothing
        Returns: nothing
        """
        self.stream.stop_stream()
        self.stream.close()
        self.p.terminate()

    def callback(self, in_data, frame_count, time_info, flag):
        """
        This function is called whenever PyAudio recieves new audio. It calls process_func to process the sound data
        and stores the result in the field "data".
        This function should never be called directly.
        Parameters: none user-exposed
        Returns: nothing of importance to the user
        """
        numpy_array = np.frombuffer(in_data, dtype=np.float32)
        start_time = time.process_time()
        self.data = self.process_func(*self.args_before, numpy_array, *self.args_after)
        end_time = time.process_time()
        elapsed_time = end_time - start_time
        if (elapsed_time > self.max_time):
            self.max_time = elapsed_time
        return None, pyaudio.paContinue

In [7]:
class DequeArray():
    def __init__(self, capacity, append_func, throw_exceptions = False):
        self.first = 0
        self.last = 0
        self.size = 0
        self.throw_exceptions = throw_exceptions
        self.capacity = capacity
        self.append_func = append_func
        self.data = []
        for i in range(self.capacity):
            self.data.append(None)
        
    def enqueue(self, data):
        if self.size == self.capacity:
            if self.throw_exceptions:
                raise AttributeError('Queue array is full')
            else:
                self.data[self.last] = data
                self.last = (self.last + 1) % self.capacity
                self.first = (self.first + 1) % self.capacity
        else:
            self.data[self.last] = data
            self.size += 1
            self.last = (self.last + 1) % self.capacity
        
    def dequeue(self):
        if self.size == 0 and self.throw_exceptions:
            if self.throw_exceptions:
                raise AttributeError('No elements to dequeue')
            else:
                return None
        else:
            temp_ref = self.data[self.first]
            self.first = (self.first + 1) % self.capacity
            return temp_ref
        
    def peek_back_n(self, n_elems):
        if n_elems < 1:
            raise ValueError('Invalid peek_n call')
        current_n = self.last
        out_elem = self.data[current_n - 1]
        for i in range(0, min(n_elems, self.capacity) - 1):
            current_n = (current_n - 1) % self.capacity
            out_elem = self.append_func(self.data[current_n - 1], out_elem)
        return out_elem

In [8]:
'''
DASThread is a thread that takes audio, stores the n most recent chunks inputted through PyAudio, and
runs a user provided process function on those chunks.
'''
class DASThread(AudioThread):
    def process_deque(self, signal):
        print("hi")
        self.audio_deque.enqueue(signal)
        self.process_func(self.audio_deque.peek_back_n(self.audio_deque.size))
        gc.collect()
        
    def append_ndarray(a: np.ndarray, b: np.ndarray):
        return np.concatenate((a, b))
    
    def __init__(self, 
                 name, 
                 process_func, 
                 starting_chunk_size = 1024, 
                 deque_array_capacity = 5):
        self.audio_deque = DequeArray(deque_array_capacity, DASThread.append_ndarray)
        self.process_func = process_func
        super().__init__(name, starting_chunk_size, self.process_deque, (), ())

In [ ]:
def process(arr: np.ndarray):
    print(arr.shape)

d = DASThread(name = "a", process_func = process)
d.start()
d.join()

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
